In [ ]:
import pymysql
import numpy as np
import skfuzzy as fuzz

# Database connection details
DB_HOST = 'localhost'
DB_USER = 'root'
DB_PASS = ''
DB_NAME = 'StudentEvaluation'

# Define weights for each category
CATEGORY_WEIGHTS = {
    "Soft Skills - Communication": 0.3,   
    "Soft Skills - Leadership": 0.1,      
    "Project": 0.2,                    
    "Sports": 0.05,                     
    "Volunteer Work": 0.1,             
    "Organization/Clubs and positions": 0.1,  
    "Events/Programs/Workshops attended": 0.1,
    "Leadership Roles": 0.05             
}

def evaluate_rules(poor_score, moderate_score, good_score, excellent_score):
    """
    Rule Evaluation: Apply explicit IF-THEN rules to determine the category label.
    """
    # Rule evaluation based on membership values
    if poor_score > max(moderate_score, good_score, excellent_score):
        return "Poor"
    elif moderate_score > max(poor_score, good_score, excellent_score):
        return "Moderate"
    elif good_score > max(poor_score, moderate_score, excellent_score):
        return "Good"
    elif excellent_score > max(poor_score, moderate_score, good_score):
        return "Excellent"
    else:
        return "Unacceptable"

def calculate_individual_score(count):
    """
    Fuzzification: Convert the crisp input 'count' into fuzzy values.
    """
    x_counts = np.arange(0, 11, 1)
    
    # Membership functions
    poor = fuzz.trimf(x_counts, [0, 0, 2])
    moderate = fuzz.trimf(x_counts, [1, 3, 5])
    good = fuzz.trimf(x_counts, [4, 6, 8])
    excellent = fuzz.trimf(x_counts, [7, 10, 10])
    
    # Fuzzification: Calculate membership values for each category
    poor_score = fuzz.interp_membership(x_counts, poor, count)
    moderate_score = fuzz.interp_membership(x_counts, moderate, count)
    good_score = fuzz.interp_membership(x_counts, good, count)
    excellent_score = fuzz.interp_membership(x_counts, excellent, count)
    
    # Rule Evaluation: Apply the explicit IF-THEN rules
    label = evaluate_rules(poor_score, moderate_score, good_score, excellent_score)
    
    # Defuzzification: Convert the count into a percentage
    percentage = int((count / 10) * 100)
    return label, percentage

def calculate_overall_fuzzy_score(category_scores):
    """
    Fuzzification for overall score, followed by rule evaluation and defuzzification.
    """
    x_scores = np.arange(0, 101, 1)
    # Membership functions for overall score
    poor = fuzz.trimf(x_scores, [0, 0, 30])
    moderate = fuzz.trimf(x_scores, [20, 50, 70])
    good = fuzz.trimf(x_scores, [60, 80, 90])
    excellent = fuzz.trimf(x_scores, [80, 100, 100])
    
    weighted_sum = 0
    total_weight = 0
    
    for category, (label, numeric_score) in category_scores.items():
        weight = CATEGORY_WEIGHTS[category]
        weighted_sum += numeric_score * weight
        total_weight += weight

    # Calculate the weighted average score
    overall_score_numeric = weighted_sum / total_weight

    # Fuzzification: Calculate membership for the overall score
    poor_level = fuzz.interp_membership(x_scores, poor, overall_score_numeric)
    moderate_level = fuzz.interp_membership(x_scores, moderate, overall_score_numeric)
    good_level = fuzz.interp_membership(x_scores, good, overall_score_numeric)
    excellent_level = fuzz.interp_membership(x_scores, excellent, overall_score_numeric)
    
    # Rule Evaluation: Determine the highest membership value
    max_level = max(poor_level, moderate_level, good_level, excellent_level)
    
    # Defuzzification: Assign the final label based on the highest membership value
    if max_level == excellent_level:
        return f"Excellent ({int(overall_score_numeric)}%)", int(overall_score_numeric)
    elif max_level == good_level:
        return f"Good ({int(overall_score_numeric)}%)", int(overall_score_numeric)
    elif max_level == moderate_level:
        return f"Moderate ({int(overall_score_numeric)}%)", int(overall_score_numeric)
    elif max_level == poor_level:
        return f"Poor ({int(overall_score_numeric)}%)", int(overall_score_numeric)
    else:
        return "Unacceptable (0%)", 0

def process_data_fuzzy():
    # Connect to the database
    conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASS, database=DB_NAME)
    cursor = conn.cursor(pymysql.cursors.DictCursor)

    # Fetch data from student_data table
    cursor.execute(" SELECT Name, Email, Telephone, `Soft Skills - Communication`, `Soft Skills - Leadership`,   Project,  Sports, `Volunteer Work`, `Organization/Clubs and positions`, `Events/Programs/Workshops attended`, `Leadership Roles` FROM student_data")
    rows = cursor.fetchall()

    for row in rows:
        name = row['Name']
        email = row['Email']
        telephone = row['Telephone']

        category_scores = {}
        detailed_scores = []
        for category in CATEGORY_WEIGHTS.keys():
            if row.get(category):
                items = [item.strip() for item in str(row[category]).split(",") if item.strip()]
                count = len(items)
                label, percentage = calculate_individual_score(count)
            else:
                items = []
                count = 0
                label, percentage = "Poor", 0

            category_scores[category] = (label, percentage)
            detailed_scores.extend([", ".join(items), count, f"{label} ({percentage}%)"])

        # Calculate the overall score using weighted fuzzy logic
        overall_score, overall_percentage = calculate_overall_fuzzy_score(category_scores)

        # Insert or update processed data into the database
        insert_values = [name, email, telephone] + detailed_scores + [overall_score]
        cursor.execute(
            f"""
            INSERT INTO processed_scores (
                Name, Email, Telephone, 
                `Soft Skills - Communication`, `Soft Skills - Communication Count`, `Soft Skills - Communication Score`,
                `Soft Skills - Leadership`, `Soft Skills - Leadership Count`, `Soft Skills - Leadership Score`,
                `Project`, `Project Count`, `Project Score`,
                `Sports`, `Sports Count`, `Sports Score`,
                `Volunteer Work`, `Volunteer Work Count`, `Volunteer Work Score`,
                `Organization/Clubs and positions`, `Organization/Clubs and positions Count`, `Organization/Clubs and positions Score`,
                `Events/Programs/Workshops attended`, `Events/Programs/Workshops attended Count`, `Events/Programs/Workshops attended Score`,
                `Leadership Roles`, `Leadership Roles Count`, `Leadership Roles Score`,
                `Overall Score`
            ) VALUES ({", ".join(["%s"] * len(insert_values))})
            ON DUPLICATE KEY UPDATE 
                `Overall Score` = VALUES(`Overall Score`)
            """,
            insert_values
        )
        conn.commit()

    print("Data processed successfully and Overall Scores stored!")
    cursor.close()
    conn.close()

# Run the process
process_data_fuzzy()
